# Train TextCNN
Stefan/Yuzhao Heng
Since Wed. Mar. 16th. 2022


## Setup
### Ipython



In [1]:
%load_ext autoreload
%autoreload 2


### Colab


In [2]:
import os
import sys


is_on_colab = 'google.colab' in sys.modules

if is_on_colab:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream
    ! pip3 install wfdb
    ! pip3 install pytorch-lightning transformers datasets
    ! pip3 install vit-pytorch

    path = os.path.join('drive', 'My Drive', 'Research', 'ECG Classification', 'ECG-Representation-Learning')
    sys.path.append(path)
    ! ls "{path}"

    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()


import torch
if torch.cuda.is_available():
    ! nvidia-smi


from icecream import ic

from ecg_transformer.util import *
ic(PATH_BASE, DIR_PROJ, PKG_NM)



ic| PATH_BASE: '/Users/stefanh/Documents/UMich/Research/ECG Classification'
    DIR_PROJ: 'ECG-Representation-Learning'
    PKG_NM: 'ecg_transformer'


('/Users/stefanh/Documents/UMich/Research/ECG Classification',
 'ECG-Representation-Learning',
 'ecg_transformer')

### code


In [3]:
from pytorch_lightning.utilities.seed import seed_everything
from icecream import ic

from ecg_transformer.models.train import get_all_setup

seed_everything(config('random-seed'))



Global seed set to 77


77

## Prep

In [4]:
# model_size = 'debug'
model_size = 'base'
# model_size = 'tiny'
t = 'original'

# n_sample = 4
# n_sample = 512
n_sample = None

with_pl = False

train_args = dict(
    num_train_epoch=32,
    train_batch_size=256,
    # train_batch_size=32,
    eval_batch_size=256*2,
    warmup_ratio=0.1,
    n_sample=n_sample,
    augment_timeout=True,
    patience=16,
    precision=16,
    log_per_epoch=True,
    # log_to_console=False
    save_while_training=True,
    save_every_n_epoch=8,
    save_top_k=4,
    tqdm=True
)
model, trainer = get_all_setup(model_size=model_size, train_args=train_args, ptbxl_type=t, with_pl=with_pl)



2022-04-19 02:10:34| [Get PTB-XL splits]::get_ptbxl_splits::ptb_dataset.py:107, INFO - Getting PTB-XL splits with n=21837... 
2022-04-19 02:10:34| [Get PTB-XL splits]::get_ptbxl_splits::ptb_dataset.py:116, INFO - Splits created with sizes {train: 17441, eval: 2193, test: 2203}... 


KeyboardInterrupt: 

## Train



In [ ]:
trainer.train()



## Evaluate



In [ ]:
# TODO
os.listdir(trainer.output_dir)
